In [1]:
import numpy as np
from tqdm import tqdm

# Deep learning: 
from keras.models import Input, Model
from keras.layers import Dense

In [2]:
import re
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict    

def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
    stop_words=['and', 'a', 'is', 'the', 'in', 'be', 'will']
    )->list:
    
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text

# Functions to find the most similar word 
def euclidean(vec1:np.array, vec2:np.array) -> float:
    """
    A function to calculate the euclidean distance between two vectors
    """
    return np.sqrt(np.sum((vec1 - vec2)**2))

def find_similar(word:str, embedding_dict:dict, top_n=10)->list:
    """
    A method to find the most similar word based on the learnt embeddings
    """
    dist_dict = {}
    word_vector = embedding_dict.get(word, [])
    if len(word_vector) > 0:
        for key, value in embedding_dict.items():
            if key!=word:
                dist = euclidean(word_vector, value)
                dist_dict.update({
                    key: dist
                })

        return sorted(dist_dict.items(), key=lambda x: x[1])[0:top_n] 

In [3]:
# Reading the text from the input folder
texts = pd.read_csv('input/sample.csv')
texts = [x for x in texts['text']]

# Defining the window for context
window = 2

<IPython.core.display.Javascript object>

In [4]:
# Creating a placeholder for the scanning of the word list
word_lists = []
all_text = []

for text in texts:

    # Cleaning the text
    text = text_preprocessing(text)

    # Appending to the all text list
    all_text += text 

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

In [5]:
unique_word_dict = create_unique_word_dict(all_text)

# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

In [6]:
for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

84it [00:00, 73507.52it/s]


In [7]:
# Converting the matrices into a sparse format because the vast majority of the data are 0s
X = np.array(X)
Y = np.array(Y)

# Defining the size of the embedding
embed_size = 8

In [8]:
# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

2022-04-27 21:17:27.667223: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
# Optimizing the network weights
model.fit(
    x=X, 
    y=Y, 
    batch_size=256,
    epochs=1000
    )

2022-04-27 21:17:28.044256: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
1/1 [==============================] - 1s 666ms/step - loss: 3.0754
Epoch 2/1000
1/1 [==============================] - 0s 23ms/step - loss: 3.0731
Epoch 3/1000
1/1 [==============================] - 0s 6ms/step - loss: 3.0709
Epoch 4/1000
1/1 [==============================] - 0s 9ms/step - loss: 3.0686
Epoch 5/1000
1/1 [==============================] - 0s 11ms/step - loss: 3.0664
Epoch 6/1000
1/1 [==============================] - 0s 15ms/step - loss: 3.0641
Epoch 7/1000
1/1 [==============================] - 0s 14ms/step - loss: 3.0619
Epoch 8/1000
1/1 [==============================] - 0s 12ms/step - loss: 3.0597
Epoch 9/1000
1/1 [==============================] - 0s 8ms/step - loss: 3.0575
Epoch 10/1000
1/1 [==============================] - 0s 8ms/step - loss: 3.0553
Epoch 11/1000
1/1 [==============================] - 0s 9ms/step - loss: 3.0531
Epoch 12/1000
1/1 [==============================] - 0s 8ms/step - loss: 3.0509
Epoch 13/1000
1/1 [=======================

1/1 [==============================] - 0s 6ms/step - loss: 2.8480
Epoch 103/1000
1/1 [==============================] - 0s 9ms/step - loss: 2.8456
Epoch 104/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.8431
Epoch 105/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.8407
Epoch 106/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.8383
Epoch 107/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.8359
Epoch 108/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.8335
Epoch 109/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.8310
Epoch 110/1000
1/1 [==============================] - 0s 94ms/step - loss: 2.8286
Epoch 111/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.8262
Epoch 112/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.8237
Epoch 113/1000
1/1 [==============================] - ETA: 0s - loss: 2.821 - 0s 9ms/step - loss: 2.8213
Epoch 114/1000
1/1

1/1 [==============================] - 0s 5ms/step - loss: 2.6096
Epoch 203/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.6073
Epoch 204/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.6050
Epoch 205/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.6028
Epoch 206/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.6005
Epoch 207/1000
1/1 [==============================] - 0s 10ms/step - loss: 2.5983
Epoch 208/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.5960
Epoch 209/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.5938
Epoch 210/1000
1/1 [==============================] - 0s 31ms/step - loss: 2.5916
Epoch 211/1000
1/1 [==============================] - 0s 8ms/step - loss: 2.5893
Epoch 212/1000
1/1 [==============================] - 0s 12ms/step - loss: 2.5871
Epoch 213/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.5849
Epoch 214/1000
1/1 [====================

1/1 [==============================] - 0s 5ms/step - loss: 2.3925
Epoch 303/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.3904
Epoch 304/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.3883
Epoch 305/1000
1/1 [==============================] - 0s 8ms/step - loss: 2.3862
Epoch 306/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.3841
Epoch 307/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.3820
Epoch 308/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.3799
Epoch 309/1000
1/1 [==============================] - 0s 8ms/step - loss: 2.3778
Epoch 310/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.3757
Epoch 311/1000
1/1 [==============================] - 0s 8ms/step - loss: 2.3736
Epoch 312/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.3716
Epoch 313/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.3695
Epoch 314/1000
1/1 [=======================

...

1/1 [==============================] - 0s 5ms/step - loss: 2.0174
Epoch 505/1000
1/1 [==============================] - 0s 3ms/step - loss: 2.0158
Epoch 506/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.0143
Epoch 507/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.0128
Epoch 508/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.0113
Epoch 509/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.0098
Epoch 510/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.0083
Epoch 511/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.0068
Epoch 512/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.0052
Epoch 513/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.0038
Epoch 514/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.0023
Epoch 515/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.0008
Epoch 516/1000
1/1 [=======================

1/1 [==============================] - 0s 6ms/step - loss: 1.8834
Epoch 605/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.8822
Epoch 606/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.8811
Epoch 607/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.8800
Epoch 608/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.8788
Epoch 609/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.8777
Epoch 610/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.8766
Epoch 611/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.8754
Epoch 612/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.8743
Epoch 613/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.8732
Epoch 614/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.8721
Epoch 615/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.8710
Epoch 616/1000
1/1 [=======================

1/1 [==============================] - 0s 5ms/step - loss: 1.7855
Epoch 706/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7847
Epoch 707/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.7839
Epoch 708/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7831
Epoch 709/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.7823
Epoch 710/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.7815
Epoch 711/1000
1/1 [==============================] - 0s 382ms/step - loss: 1.7807
Epoch 712/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7799
Epoch 713/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.7792
Epoch 714/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7784
Epoch 715/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7776
Epoch 716/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7768
Epoch 717/1000
1/1 [=====================

1/1 [==============================] - 0s 4ms/step - loss: 1.7179
Epoch 806/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7174
Epoch 807/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7168
Epoch 808/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.7163
Epoch 809/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.7157
Epoch 810/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7152
Epoch 811/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.7146
Epoch 812/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.7141
Epoch 813/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7135
Epoch 814/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7130
Epoch 815/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7124
Epoch 816/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.7119
Epoch 817/1000
1/1 [=======================

1/1 [==============================] - 0s 5ms/step - loss: 1.6696
Epoch 907/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.6692
Epoch 908/1000
1/1 [==============================] - 0s 8ms/step - loss: 1.6688
Epoch 909/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.6684
Epoch 910/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6680
Epoch 911/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6676
Epoch 912/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6672
Epoch 913/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6668
Epoch 914/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6664
Epoch 915/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.6660
Epoch 916/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6656
Epoch 917/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6652
Epoch 918/1000
1/1 [=======================

In [11]:
# Obtaining the weights from the neural network. 
# These are the so called word embeddings

# The input layer 
weights = model.get_weights()[0]

In [12]:
weights

array([[ 3.5583395e-01,  1.3214129e+00, -1.1853784e+00,  1.1436698e+00,
         8.8958806e-01,  3.2360053e-01, -3.2460424e-01,  4.1349709e-01],
       [-2.2411484e-01, -3.3993453e-01, -2.4204421e-01, -5.6281990e-01,
         1.3052272e+00,  3.2089573e-01,  5.7656074e-01, -7.7915442e-01],
       [-1.2429678e+00,  1.8633750e-01,  7.2254491e-01,  4.6909553e-01,
         6.3713647e-02,  1.1592378e-01,  3.4830931e-01, -4.8824036e-01],
       [ 8.2414258e-01, -1.2062601e+00,  3.9717138e-01, -1.4997213e-01,
         6.1817050e-01, -7.7137870e-01, -1.2533088e+00,  6.2784767e-01],
       [ 9.1442353e-01,  4.7307703e-01, -1.2520542e+00,  1.3448410e+00,
         1.0053205e+00,  4.5885885e-01, -6.0287392e-01,  1.0238960e+00],
       [-6.4149940e-01, -1.3052942e+00,  7.2688317e-01, -5.1954109e-02,
        -9.6609169e-01,  3.7290803e-01, -1.0136725e+00, -4.1071871e-01],
       [-7.1249062e-01,  8.8558149e-01, -2.5237891e-01, -8.3597040e-01,
         6.2330461e-01,  1.3128392e+00, -7.5913990e-01,  5

In [13]:
# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

In [14]:
embedding_dict

{'beautiful': array([ 0.35583395,  1.3214129 , -1.1853784 ,  1.1436698 ,  0.88958806,
         0.32360053, -0.32460424,  0.4134971 ], dtype=float32),
 'boy': array([-0.22411484, -0.33993453, -0.24204421, -0.5628199 ,  1.3052272 ,
         0.32089573,  0.57656074, -0.7791544 ], dtype=float32),
 'can': array([-1.2429678 ,  0.1863375 ,  0.7225449 ,  0.46909553,  0.06371365,
         0.11592378,  0.3483093 , -0.48824036], dtype=float32),
 'children': array([ 0.8241426 , -1.2062601 ,  0.39717138, -0.14997213,  0.6181705 ,
        -0.7713787 , -1.2533088 ,  0.6278477 ], dtype=float32),
 'daughter': array([ 0.9144235 ,  0.47307703, -1.2520542 ,  1.344841  ,  1.0053205 ,
         0.45885885, -0.6028739 ,  1.023896  ], dtype=float32),
 'family': array([-0.6414994 , -1.3052942 ,  0.7268832 , -0.05195411, -0.9660917 ,
         0.37290803, -1.0136725 , -0.4107187 ], dtype=float32),
 'future': array([-0.7124906,  0.8855815, -0.2523789, -0.8359704,  0.6233046,
         1.3128392, -0.7591399,  0.5481

In [15]:
find_similar(word='children', embedding_dict=embedding_dict, top_n=3)

[('their', 2.2238736), ('son', 2.4032679), ('family', 2.6891553)]